#### This Program Downloads the historical stock price and index data from sharesansar.com and merges to the existing excel csv file. The program automatically identifies the most recent available date and downloads the remaining data.

### Checking For Latest Available Data on PriceHistory.csv

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv(r'PriceHistory.csv', parse_dates= ['Date'],  thousands= ',')
data.sort_values(by = 'Date', ascending = True, inplace = True)
latest_data_available = data['Date'].iloc[-1]
print(latest_data_available.date())

### Date To Scrape From Sharesansar

In [ ]:
date_to_start_scraping_from  = latest_data_available.date() + pd.to_timedelta(1, unit="D")
dates = pd.date_range(start = date_to_start_scraping_from, end = pd.to_datetime("today"))
dates = [d for d in dates if not d.isoweekday() in [5,6]] # List excluding Friday and Saturday
print(dates)

In [ ]:
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time
import os
import calendar
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get('https://www.sharesansar.com/today-share-price')

blank_df = []
for date in dates:
    driver.find_element(By.XPATH,'//*[@id="fromdate"]').clear()
    driver.find_element(By.XPATH,'//*[@id="fromdate"]').send_keys(f'{date.date()}') 
    driver.find_element(By.XPATH,'//*[@id="btn_todayshareprice_submit"]').click()
    time.sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    todays_price = soup.find('table', id = 'headFixed')
    if len(todays_price.findAll('tr'))<10:
        pass
    else:
        output_rows = []
        for table_row in todays_price.findAll('tr'): # looping through all the table rows in a page
            columns = table_row.findAll('td') #finding the cell values of every row in the table
            output_row = []
            for column in columns: # looping through each cellvalue data in a row 
                output_row.append(column.text)
            output_rows.append(output_row)
            headers_list = []
            for headers in todays_price.find_all('th'):
                headers_list.append(headers.text)
        todays_price_dataframe = pd.DataFrame(output_rows)[1:]
        todays_price_dataframe.columns = headers_list
        todays_price_dataframe['Symbol'] = todays_price_dataframe['Symbol'].str.replace("\n", "")
        todays_price_dataframe.set_index('S.No', inplace = True)
        todays_price_dataframe['Date'] = date.date()
        blank_df.append(todays_price_dataframe)
        print(date.date())

driver.quit()


In [ ]:
df = pd.concat(blank_df)
df['Date']  = df['Date'].astype('datetime64')
df.set_index('Date', inplace = True)
df = df[['Symbol','Open','High', 'Low', 'Close','Vol', 'Turnover','Trans.']]
columns_except_symbol = df.columns.difference(['Symbol'])
df[columns_except_symbol]= df[columns_except_symbol].replace(',',  '', regex= True).astype(float)
last_traded_day = df.index[-1]
print(last_traded_day.date())

In [ ]:
with open('PriceHistory.csv', 'a') as f:
    df.to_csv(f, header = False, lineterminator='\n')
print('Completed')

### Download Index From Sharesansar

In [ ]:
sharesansar_sectors= ['Nepse Index','Banking SubIndex',	'Development Bank Index',	'Finance Index',	'Hotels And Tourism',	'HydroPower Index',	'Investment',	'Life Insurance',	'Manufacturing And Processing',	'Microfinance Index',	'Mutual Fund',	'Non Life Insurance',	'Others Index',	'Trading Index']

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import time

options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get('https://www.sharesansar.com/index-history-data')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


driver.find_element(By.ID,'fromDate').clear()
driver.find_element(By.ID,"fromDate").send_keys(f"{dates[0].date()}") 
driver.find_element(By.ID,'toDate').clear()
driver.find_element(By.ID,'toDate').send_keys(f'{last_traded_day.date()}') #f'{last_traded_day.date()}'



sectors_df = []
for each_sector in sharesansar_sectors:
    
    driver.find_element(By.ID,"select2-index-container").click()
    driver.find_element(By.CSS_SELECTOR,"input[class='select2-search__field']").send_keys(each_sector)

    driver.find_element(By.ID, "btn_indxhis_submit").click()
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    output_rows = []
    for table_row in soup.findAll('tr'): # looping through all the table rows in a page
        columns = table_row.findAll('td') #finding the cell values of every row in the table
        output_row = []
        for column in columns: # looping through each cellvalue data in a row 
            output_row.append(column.text)
        output_rows.append(output_row)
        headers_list = []
        for headers in soup.find_all('th'):
            headers_list.append(headers.text)

    index_data = pd.DataFrame(output_rows)[1:]
    index_data.columns = headers_list
    index_data.insert(loc = 0, column = 'Ticker', value  = each_sector)
    index_data['Date']  = index_data['Date'].astype('datetime64')
    index_data.set_index('Date', inplace = True)
    columns_except_symbol = index_data.columns.difference(['Ticker'])
    index_data[columns_except_symbol]= index_data[columns_except_symbol].replace(',',  '', regex= True).astype(float)
    index_data = index_data.drop(['S.N.', 'Change', 'Per Change (%)'], axis = 1)
    time.sleep(1)
    sectors_df.append(index_data)
    print(each_sector)
driver.quit()

In [ ]:
daily_indices_data = pd.concat(sectors_df)
with open('PriceHistory.csv', 'a') as f:
    daily_indices_data.to_csv(f, header = False, lineterminator='\n')
print('*Completed*')